In [1]:
import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from btgym.a3c import Launcher, BaseLSTMPolicy

In [2]:
import tensorflow as tf

class StatePreProcessor():
    def __init__(self, x, window, stride, pooling_type='MAX', padding='SAME'):
        """_____"""
        self.x = x
        
        x_max = tf.expand_dims(self.x[...,0], [-1])
        
        x_min = tf.negative(
            tf.expand_dims(self.x[...,-1], [-1])
        )
        # MAX H-values pooling:
        x_max = tf.nn.pool(
            input=x_max,
            window_shape=[window],
            pooling_type=pooling_type,
            padding=padding,
            dilation_rate=None,
            strides=[stride],
        )
        # MIN L-values pooling:
        x_min = tf.nn.pool(
            input=x_min,
            window_shape=[window],
            pooling_type=pooling_type,
            padding=padding,
            dilation_rate=None,
            strides=[stride],
        )
        
        self.output = tf.concat([x_max, x_min], axis=2)
    
    def process(self, x):
        sess = tf.get_default_session()
        return sess.run(self.output, {self.x: x})

In [3]:

import tensorflow.contrib.rnn as rnn

class LSTMPolicy2(BaseLSTMPolicy):
    """
    +++++
    """
    def __init__(self, ob_space, ac_space,
                 lstm_class=rnn.BasicLSTMCell, lstm_layers=(256,)):
        
        self.x = x = tf.placeholder(tf.float32, [None] + list(ob_space), name='x_in_pl')
        
        pre_processor = StatePreProcessor(x, 20, 2)
        
        #x_pooled = tf.contrib.layers.flatten(pre_processor.output)
        
        x_pooled = pre_processor.output
        
        #print('GOT HERE 1')
        print('x_pooled:', x_pooled.shape)
        # Run LSTM along rollout time dimension and evrything else:
        super(LSTMPolicy2, self).__init__(x_pooled, ob_space, ac_space, lstm_class, lstm_layers)

        

In [ ]:
#tf.reset_default_graph()
#pi = LSTMPolicy2((10,2),(3))

In [6]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        self.order_penalty = 1
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):    
        if trade.isclosed:
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
        
    def get_state(self):
        T =2e3
        self.state['raw_state'] = self.raw_state
        #self.state['model_input'] = (self.raw_state[:, 1:3] - self.raw_state[0,0]) * 2e2
        self.state['model_input'] = np.gradient(self.raw_state[:, 1:3], axis=1) * T
        
        return self.state
    
    def get_reward(self):
        """
        Defines reward as function of last closed trade result with penalty for erronious order placement.
        """
        # Low-value term:
        r = (self.broker.get_value() / self.env.broker.startingcash - 1) * 10
        
        # Result (main-value):
        if self.trade_just_closed:
            r += self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            r -= self.order_penalty
            self.order_failed = False

        return r / 20  # sinse reward is just quick-example, denominator is here to keep gradients sane

# Set backtesting engine parameters:

time_embed_dim = 180

state_shape = {
    'raw_state': spaces.Box(low=-1, high=1, shape=(time_embed_dim, 4)),
    'model_input': spaces.Box(low=-100, high=100, shape=(time_embed_dim, 2))
}

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    MyStrategy,
    state_shape=state_shape,
    portfolio_actions=('hold', 'buy', 'sell'),
    drawdown_call=5, # max to loose, in percent of initial cash
    target_call=10,  # max to win, same
    skip_frame=10,
)

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)

MyCerebro.addanalyzer(bt.analyzers.DrawDown)

MyDataset = BTgymDataset(
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    filename='../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    #filename='../examples/data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=55,
    start_00=False,
    time_gap_hours=8,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=False,
    render_ylabel='HL avg',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)

# Set tensorflow distributed cluster and a3c configuration:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_test',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    policy_class=LSTMPolicy2,
    policy_config={'lstm_layers': (256,)},
    rollout_length=20,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=10,
    verbose=2
    
)

[2017-09-04 16:49:02,636] Launcher ready.


In [ ]:
launcher.run()

[2017-09-04 16:49:02,920] parameters_server started.
[2017-09-04 16:49:02,922] worker_0 tf.server started.
[2017-09-04 16:49:02,925] making environment.
[2017-09-04 16:49:02,927] worker_0 is data_master: True
[2017-09-04 16:49:05,032] worker_0:envronment ok.
[2017-09-04 16:49:05,035] A3C_0: init() started


x_pooled: (?, 90, 2)
x_pooled: (?, 90, 2)


[2017-09-04 16:49:06,367] A3C_0: train op defined
[2017-09-04 16:49:06,422] A3C_0: init() done
[2017-09-04 16:49:06,423] worker_0:trainer ok.
[2017-09-04 16:49:07,057] worker_0: connecting to the parameter server... 


INFO:tensorflow:Restoring parameters from ./tmp/a3c_test/train/model.ckpt-329997


[2017-09-04 16:49:07,086] Restoring parameters from ./tmp/a3c_test/train/model.ckpt-329997
[2017-09-04 16:49:07,936] Press `Ctrl-C` to stop training and close launcher.
[2017-09-04 16:49:07,939] worker_3 tf.server started.
[2017-09-04 16:49:07,934] worker_2 tf.server started.
[2017-09-04 16:49:07,930] worker_1 tf.server started.
[2017-09-04 16:49:07,947] making environment.
[2017-09-04 16:49:07,947] worker_4 tf.server started.
[2017-09-04 16:49:07,948] worker_5 tf.server started.
[2017-09-04 16:49:07,952] making environment.
[2017-09-04 16:49:07,952] making environment.
[2017-09-04 16:49:07,953] worker_3 is data_master: False
[2017-09-04 16:49:07,956] making environment.
[2017-09-04 16:49:07,957] making environment.
[2017-09-04 16:49:07,953] worker_6 tf.server started.
[2017-09-04 16:49:07,960] worker_1 is data_master: False
[2017-09-04 16:49:07,960] worker_2 is data_master: False
[2017-09-04 16:49:07,958] worker_7 tf.server started.
[2017-09-04 16:49:07,962] making environment.
[2017-

...

[2017-09-05 11:39:10,168] Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


[2017-09-05 11:44:10,161] Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


[2017-09-05 11:49:10,165] Saving checkpoint to path ./tmp/a3c_test/train/model.ckpt


In [ ]:
env_config.update({'port': 5050, 'data_port': 5049})
env = BTgymEnv(**env_config)

In [ ]:
o = env.reset()

In [ ]:
o

In [ ]:
o,r,d,i = env.step(0)
print(o)

In [ ]:
env.close()